# BERTGCN with  PCA 

In [1]:
import pandas as pd
# data= pd.read_csv("../dataset/yelp/yelp_over512.csv",names=['label','text'])
data= pd.read_csv("../dataset/IMDB/IMDB_over512.csv",names=['label','text'])

print(len(data['label'])*0.5)

2005.5


In [4]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
#dataset_name = 'own'
#sentences = ['Would you like a plain sweater or something else?', 'Great. We have some very nice wool slacks over here. Would you like to take a look?']
#labels = ['Yes' , 'No' ]
#train_or_test_list = ['train', 'test']
import pandas as pd
# data= pd.read_csv("../dataset/20ng/20ng_over512.csv", names = ["label", "text"])
# data= pd.read_csv("../dataset/hyper/hy512.csv",names=['label','text'])
# data= pd.read_csv("../dataset/yelp/yelp_over512.csv",names=['label','text'])

data= pd.read_csv("../dataset/IMDB/IMDB_over512.csv",names=['label','text'])
LE = LabelEncoder()
data['label'] = LE.fit_transform(data['label'])
sentences = []
labels = []
train_or_test_list = []
for index,  label,text in data.itertuples():
  sentences.append(text)
  labels.append(label)
  train_or_test_list.append('train' if index < 2005 else 'test')
dataset_name = 'imdb'


meta_data_list = []

for i in range(len(sentences)):
    meta = str(i) + '\t' + train_or_test_list[i] + '\t' + str(labels[i])
    meta_data_list.append(meta)

meta_data_str = '\n'.join(meta_data_list)

f = open( 'data/ '+ dataset_name + '.txt', 'w')
f.write(meta_data_str)
f.close()

#corpus_str = '\n'.join(sentences)
corpus_str = '\n'.join([str(i) for i in sentences])

f = open('data/corpus/' + dataset_name + '.txt', 'w')
f.write(corpus_str)
f.close()

In [5]:
from nltk.corpus import stopwords
import nltk
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
dataset = 'imdb'
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
remove_limit = 5
data= pd.read_csv("../dataset/IMDB/IMDB_over512.csv",names=['label','text'])

# data= pd.read_csv("../dataset/20ng/20ng_over512.csv", names = ["label", "text"])
# data= pd.read_csv("../dataset/hyper/hy512.csv",names=['label','text'])
sentences = list(data['text'])

def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

# original_word_freq = {}  # to remove rare words
# fi = open('data/corpus/clean_' + dataset + '.txt', 'w', encoding="utf-8")
# for sentence in sentences:
#     temp = clean_str(sentence)
#     fi.write(temp)
#     fi.close()
with open('data/corpus/' + dataset + '.txt', 'w', encoding="utf-8") as f:
    for sentence in sentences:
        temp = clean_str(sentence)
        f.write(temp+"\n")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nm6104054/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
f = open('data/' + dataset + '.txt', 'r', encoding="utf-8")


In [7]:
import os
import sys
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import pandas as pd
import numpy as np
from math import log
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder

dataset = 'imdb'
# data= pd.read_csv("../dataset/20ng/20ng_over512.csv", names = ["label", "text"])
# data= pd.read_csv("../dataset/yelp/yelp_over512.csv",names=['label','text'])
data= pd.read_csv("../dataset/IMDB/IMDB_over512.csv",names=['label','text'])

# data= pd.read_csv("../dataset/hyper/hy512.csv",names=['label','text'])
# LE = LabelEncoder()
data['label'] = LE.fit_transform(data['label'])
sentences = []
labels = []
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

word_embeddings_dim = 300
word_vector_map = {}

doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open('data/' + dataset + '.txt', 'r', encoding="utf-8")
lines = f.readlines()
for line in lines:
    doc_name_list.append(line.strip())
    temp = line.split("\t")
    if temp[1].find('test') != -1:
        doc_test_list.append(line.strip())
    elif temp[1].find('train') != -1:
        doc_train_list.append(line.strip())
f.close()

doc_content_list = []
f = open('data/corpus/' + dataset + '.txt', 'r', encoding="utf-8")
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()

train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
random.shuffle(train_ids)

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('data/' + dataset + '.train.index', 'w')
f.write(train_ids_str)
f.close()

test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('data/' + dataset + '.test.index', 'w')
f.write(test_ids_str)
f.close()

shuffle_doc_name_list = []
shuffle_doc_words_list = []
ids = train_ids + test_ids
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('data/' + dataset + '_shuffle.txt', 'w', encoding="utf-8")
f.write(shuffle_doc_name_str)
f.close()

f = open('data/corpus/' + dataset + '_shuffle.txt', 'w', encoding="utf-8")
f.write(shuffle_doc_words_str)
f.close()

word_freq = {}
word_set = set()
for doc_words in shuffle_doc_words_list:
    words = tokenizer.tokenize(doc_words)
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_doc_list = {}

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = tokenizer.tokenize(doc_words)
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)

word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

vocab_str = '\n'.join(vocab)

f = open('data/corpus/' + dataset + '_vocab.txt', 'w',encoding="utf-8")
f.write(vocab_str)
f.close()

label_set = set()
for doc_meta in shuffle_doc_name_list:
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

label_list_str = '\n'.join(label_list)
f = open('data/corpus/' + dataset + '_labels.txt', 'w',encoding="utf-8")
f.write(label_list_str)
f.close()

train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size

real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('data/' + dataset + '.real_train.name', 'w',encoding="utf-8")
f.write(real_train_doc_names_str)
f.close()

row_x = []
col_x = []
data_x = []

for i in range(real_train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = tokenizer.tokenize(doc_words)
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        data_x.append(doc_vec[j] / doc_len)
        
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))

y = []
for i in range(real_train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)

test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in range(test_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = tokenizer.tokenize(doc_words)
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        data_tx.append(doc_vec[j] / doc_len)

tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))

ty = []
for i in range(test_size):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)

word_vectors = np.random.uniform(-0.01, 0.01, (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

row_allx = []
col_allx = []
data_allx = []

for i in range(train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = tokenizer.tokenize(doc_words)
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        data_allx.append(doc_vec[j] / doc_len)
for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)

allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))

ally = []
for i in range(train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

for i in range(vocab_size):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)

window_size = 20
windows = []

for doc_words in shuffle_doc_words_list:
    words = tokenizer.tokenize(doc_words)
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)


word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

row = []
col = []
weight = []

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

doc_word_freq = {}

for doc_id in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[doc_id]
    words = tokenizer.tokenize(doc_words)
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = tokenizer.tokenize(doc_words)
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))

f = open("data/ind.{}.x".format(dataset), 'wb')
pkl.dump(x, f)
f.close()

f = open("data/ind.{}.y".format(dataset), 'wb')
pkl.dump(y, f)
f.close()

f = open("data/ind.{}.test_x".format(dataset), 'wb')
pkl.dump(tx, f)
f.close()

f = open("data/ind.{}.test_y".format(dataset), 'wb')
pkl.dump(ty, f)
f.close()

f = open("data/ind.{}.all_x".format(dataset), 'wb')
pkl.dump(allx, f)
f.close()

f = open("data/ind.{}.all_y".format(dataset), 'wb')
pkl.dump(ally, f)
f.close()

f = open("data/ind.{}.adj".format(dataset), 'wb')
pkl.dump(adj, f)
f.close()

Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors


(1350, 300) (1350, 2) (2511, 300) (2511, 2) (27026, 300) (27026, 2)


In [11]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import os
import sys
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import pandas as pd
import numpy as np
from math import log
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder

dataset = 'yelp'

def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def load_data(dataset_str):
    names = ['x', 'y', 'test_x', 'test_y', 'all_x', 'all_y', 'graph']
    
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            objects.append(pkl.load(f))

    x, y, test_x, test_y, all_x, all_y, graph = tuple(objects)
    test_idx_reorder = parse_index_file(
        "data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)
    print(x.shape, y.shape, test_x.shape, test_y.shape, all_x.shape, all_y.shape)

    features = sp.vstack((all_x, test_x)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((all_y, test_y))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train, y_val, y_test = np.zeros(labels.shape), np.zeros(labels.shape), np.zeros(labels.shape)
    
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask


def load_corpus(dataset_str):
    names = ['x', 'y', 'test_x', 'test_y', 'all_x', 'all_y', 'adj']
    
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            objects.append(pkl.load(f))

    x, y, test_x, test_y, all_x, all_y, adj = tuple(objects)
    print(x.shape, y.shape, test_x.shape, test_y.shape, all_x.shape, all_y.shape)

    features = sp.vstack((all_x, test_x)).tolil()
    labels = np.vstack((all_y, test_y))
    print(len(labels))

    train_idx_orig = parse_index_file(
        "data/{}.train.index".format(dataset_str))
    train_size = len(train_idx_orig)

    val_size = train_size - x.shape[0]
    test_size = test_x.shape[0]

    idx_train = range(len(y))
    idx_val = range(len(y), len(y) + val_size)
    idx_test = range(all_x.shape[0], all_x.shape[0] + test_size)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train, y_val, y_test = np.zeros(labels.shape), np.zeros(labels.shape), np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size

def sparse_to_tuple(sparse_mx):
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def preprocess_features(features):
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)

def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def preprocess_adj(adj):
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)


def normalize_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

# GCN

In [25]:
import torch
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.utils.data as Data
from transformers import AutoModel, AutoTokenizer
from utils import *
import dgl
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os
import shutil
import argparse
import sys
import logging
from datetime import datetime
from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer, Engine
from ignite.metrics import Accuracy, Loss, Metric
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.utils import manual_seed
# from model import BertGCN
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from dgl.nn.pytorch import GraphConv
from dgl import function as fn
from dgl.base import DGLError
from dgl.utils import expand_as_pair
from transformers import AutoModel, AutoTokenizer
from dgl.nn.pytorch import GraphConv
from dgl import function as fn
from dgl.base import DGLError
from dgl.utils import expand_as_pair
from dgl.nn import GATConv
from transformers import RobertaTokenizer

# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
class BertClassifier(torch.nn.Module):
    def __init__(self, pretrained_model:str ="roberta-base", nb_class:int = 8):
        super(BertClassifier, self).__init__()
        self.nb_class = nb_class
        # self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.bert_model = AutoModel.from_pretrained(pretrained_model)
        self.feat_dim = list(self.bert_model.modules())[-2].out_features
        self.classifier = torch.nn.Linear(self.feat_dim, nb_class)

    def forward(self, input_ids, attention_mask):
        cls_feats = self.bert_model(input_ids, attention_mask)[0][:, 0]
        cls_logit = self.classifier(cls_feats)
        return cls_logit


class BertGCN(torch.nn.Module):
    def __init__(self, pretrained_model:str ="roberta-base", nb_class:int = 2, ratio:float = 0.7, gcn_layers:int = 2, n_hidden:int = 200, dropout:int = 0.5):
        super(BertGCN, self).__init__()
        self.ratio = ratio
        self.nb_class = nb_class
        # self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.bert_model = AutoModel.from_pretrained(pretrained_model)
        self.feat_dim = list(self.bert_model.modules())[-2].out_features
        self.classifier = torch.nn.Linear(self.feat_dim, nb_class)
        self.gcn = GCN(
            in_feats=self.feat_dim,
            n_hidden=n_hidden,
            n_classes=nb_class,
            n_layers=gcn_layers-1,
            activation=F.elu,
            dropout=dropout
        )

    def forward(self, g, idx):
        input_ids, attention_mask = g.ndata['input_ids'][idx], g.ndata['attention_mask'][idx]
        if self.training:
            cls_feats = self.bert_model(input_ids, attention_mask)[0][:, 0]
            g.ndata['cls_feats'][idx] = cls_feats
        else:
            cls_feats = g.ndata['cls_feats'][idx]
        cls_logit = self.classifier(cls_feats)
        cls_pred = torch.nn.Softmax(dim=1)(cls_logit)
        gcn_logit = self.gcn(g.ndata['cls_feats'], g, g.edata['edge_weight'])[idx]
        gcn_pred = torch.nn.Softmax(dim=1)(gcn_logit)
        pred = (gcn_pred+1e-10) * self.ratio + cls_pred * (1 - self.ratio)
        pred = torch.log(pred)
        return pred

    
class BertGAT(torch.nn.Module):
    def __init__(self, pretrained_model='roberta_base', nb_class=2, m=0.7, gcn_layers=2, heads=8, n_hidden=32, dropout=0.5):
        super(BertGAT, self).__init__()
        self.m = m
        self.nb_class = nb_class
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.bert_model = AutoModel.from_pretrained(pretrained_model)
        self.feat_dim = list(self.bert_model.modules())[-2].out_features
        self.classifier = torch.nn.Linear(self.feat_dim, nb_class)
        self.gcn = GAT(
                 num_layers=gcn_layers-1,
                 in_dim=self.feat_dim,
                 num_hidden=n_hidden,
                 num_classes=nb_class,
                 heads=[heads] * (gcn_layers-1) + [1],
                 activation=F.elu,
                 feat_drop=dropout,
                 attn_drop=dropout,
        )

    def forward(self, g, idx):
        input_ids, attention_mask = g.ndata['input_ids'][idx], g.ndata['attention_mask'][idx]
        if self.training:
            cls_feats = self.bert_model(input_ids, attention_mask)[0][:, 0]
            g.ndata['cls_feats'][idx] = cls_feats
        else:
            cls_feats = g.ndata['cls_feats'][idx]
        cls_logit = self.classifier(cls_feats)
        cls_pred = torch.nn.Softmax(dim=1)(cls_logit)
        gcn_logit = self.gcn(g.ndata['cls_feats'], g)[idx]
        gcn_pred = torch.nn.Softmax(dim=1)(gcn_logit)
        pred = (gcn_pred+1e-10) * self.m + cls_pred * (1 - self.m)
        pred = torch.log(pred)
        return pred    


class GAT(torch.nn.Module):
    def __init__(self,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop=0,
                 attn_drop=0,
                 negative_slope=0.2,
                 residual=False
    ):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = torch.nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l-1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, inputs, g):
        h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](g, h).flatten(1)
        # output projection
        logits = self.gat_layers[-1](g, h).mean(1)
        return logits
    
class GCN(torch.nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 normalization='none'):
        super(GCN, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GraphConvEdgeWeight(in_feats, n_hidden, activation=activation, norm=normalization))
        for i in range(n_layers - 1):
            self.layers.append(GraphConvEdgeWeight(n_hidden, n_hidden, activation=activation, norm=normalization))
        self.layers.append(GraphConvEdgeWeight(n_hidden, n_classes, norm=normalization))
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, features, g, edge_weight):
        h = features
        for i, layer in enumerate(self.layers):
            if i != 0:
                h = self.dropout(h)
            h = layer(g, h, edge_weights=edge_weight)
        return h
    
class GraphConvEdgeWeight(GraphConv):

    def forward(self, graph, feat,  weight=None, edge_weights=None):
        with graph.local_scope():
            if not self._allow_zero_in_degree:
                if (graph.in_degrees() == 0).any():
                    raise DGLError('There are 0-in-degree nodes in the graph, '
                                   'output for those nodes will be invalid. '
                                   'This is harmful for some applications, '
                                   'causing silent performance regression. '
                                   'Adding self-loop on the input graph by '
                                   'calling `g = dgl.add_self_loop(g)` will resolve '
                                   'the issue. Setting ``allow_zero_in_degree`` '
                                   'to be `True` when constructing this module will '
                                   'suppress the check and let the code run.')

            feat_src, feat_dst = expand_as_pair(feat, graph)
            if self._norm == 'both':
                degs = graph.out_degrees().float().clamp(min=1)
                norm = torch.pow(degs, -0.5)
                shp = norm.shape + (1,) * (feat_src.dim() - 1)
                norm = torch.reshape(norm, shp)
                feat_src = feat_src * norm

            if weight is not None:
                if self.weight is not None:
                    raise DGLError('External weight is provided while at the same time the'
                                   ' module has defined its own weight parameter. Please'
                                   ' create the module with flag weight=False.')
            else:
                weight = self.weight

            if self._in_feats > self._out_feats:
                if weight is not None:
                    feat_src = torch.matmul(feat_src, weight)
                graph.srcdata['h'] = feat_src
                if edge_weights is None:
                    graph.update_all(fn.copy_src(src='h', out='m'),
                                     fn.sum(msg='m', out='h'))
                else:
                    graph.edata['a'] = edge_weights
                    graph.update_all(fn.u_mul_e('h', 'a', 'm'),
                                     fn.sum(msg='m', out='h'))
                rst = graph.dstdata['h']
            else:
                graph.srcdata['h'] = feat_src
                if edge_weights is None:
                    graph.update_all(fn.copy_src(src='h', out='m'),
                                     fn.sum(msg='m', out='h'))
                else:
                    graph.edata['a'] = edge_weights
                    graph.update_all(fn.u_mul_e('h', 'a', 'm'),
                                     fn.sum(msg='m', out='h'))
                rst = graph.dstdata['h']
                if weight is not None:
                    rst = torch.matmul(rst, weight)

            if self._norm != 'none':
                degs = graph.in_degrees().float().clamp(min=1)
                if self._norm == 'both':
                    norm = torch.pow(degs, -0.5)
                else:
                    norm = 1.0 / degs
                shp = norm.shape + (1,) * (feat_dst.dim() - 1)
                norm = torch.reshape(norm, shp)
                rst = rst * norm

            if self.bias is not None:
                rst = rst + self.bias

            if self._activation is not None:
                rst = self._activation(rst)

            return rst

In [26]:
seed = 51
manual_seed(seed)
max_length = 64
batch_size = 256
ratio = 0.1
nb_epochs = 10
dataset = 'yelp'
n_hidden = 200
dropout = 0.5
gcn_lr = 1e-3
bert_lr = 1e-5
args = [max_length, batch_size, ratio, nb_epochs, dataset, n_hidden, dropout, gcn_lr, bert_lr, seed]

In [27]:
ckpt_dir = './checkpoint/robertagcn_{}_{}'.format(dataset, ratio)
os.makedirs(ckpt_dir, exist_ok=True)

streamhandle = logging.StreamHandler(sys.stdout)
streamhandle.setFormatter(logging.Formatter('%(message)s'))
streamhandle.setLevel(logging.INFO)

filehandle = logging.FileHandler(filename=os.path.join(ckpt_dir, 'training.log'), mode='w')
filehandle.setFormatter(logging.Formatter('%(message)s'))
filehandle.setLevel(logging.INFO)

logger = logging.getLogger('training logger')
logger.addHandler(streamhandle)
logger.addHandler(filehandle)
logger.setLevel(logging.INFO)

cpu = torch.device('cpu')
gpu = torch.device('cuda:0')

logger.info('params:')
logger.info(str(args))
logger.info('checkpoints path: {}'.format(ckpt_dir))

params:
params:
params:
[64, 256, 0.1, 10, 'yelp', 200, 0.5, 0.001, 1e-05, 51]
[64, 256, 0.1, 10, 'yelp', 200, 0.5, 0.001, 1e-05, 51]
[64, 256, 0.1, 10, 'yelp', 200, 0.5, 0.001, 1e-05, 51]
checkpoints path: ./checkpoint/robertagcn_yelp_0.1
checkpoints path: ./checkpoint/robertagcn_yelp_0.1
checkpoints path: ./checkpoint/robertagcn_yelp_0.1


In [28]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(dataset)

nb_node = features.shape[0]
nb_train, nb_val, nb_test = train_mask.sum(), val_mask.sum(), test_mask.sum()
nb_word = nb_node - nb_train - nb_val - nb_test
nb_class = y_train.shape[1]

# model = BertGAT( pretrained_model='roberta-base', nb_class=2, m=0.7, gcn_layers=2, heads=8, n_hidden=32, dropout=0.5)
# 
model = BertGCN(nb_class=nb_class, ratio=ratio, n_hidden=n_hidden, dropout=dropout)

corpse_file = './data/corpus/' + dataset +'_shuffle.txt'
with open(corpse_file, 'r', encoding="utf-8") as f:
    text = f.read()
    text = text.replace('\\', '')
    text = text.split('\n')

def encode_input(text, tokenizer):
    input = tokenizer(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')
    return input.input_ids, input.attention_mask


input_ids, attention_mask = encode_input(text, model.tokenizer)
input_ids = torch.cat([input_ids[:-nb_test], torch.zeros((nb_word, max_length), dtype=torch.long), input_ids[-nb_test:]])
attention_mask = torch.cat([attention_mask[:-nb_test], torch.zeros((nb_word, max_length), dtype=torch.long), attention_mask[-nb_test:]])

y = y_train + y_test + y_val
y_train = y_train.argmax(axis=1)
y = y.argmax(axis=1)

doc_mask  = train_mask + val_mask + test_mask

adj_norm = normalize_adj(adj + sp.eye(adj.shape[0]))
g = dgl.from_scipy(adj_norm.astype('float32'), eweight_name='edge_weight')
g.ndata['input_ids'], g.ndata['attention_mask'] = input_ids, attention_mask
g.ndata['label'], g.ndata['train'], g.ndata['val'], g.ndata['test'] = \
    torch.LongTensor(y), torch.FloatTensor(train_mask), torch.FloatTensor(val_mask), torch.FloatTensor(test_mask)
g.ndata['label_train'] = torch.LongTensor(y_train)
g.ndata['cls_feats'] = torch.zeros((nb_node, model.feat_dim))

logger.info('graph information:')
logger.info(str(g))

train_idx = Data.TensorDataset(torch.arange(0, nb_train, dtype=torch.long))
val_idx = Data.TensorDataset(torch.arange(nb_train, nb_train + nb_val, dtype=torch.long))
test_idx = Data.TensorDataset(torch.arange(nb_node-nb_test, nb_node, dtype=torch.long))
doc_idx = Data.ConcatDataset([train_idx, val_idx, test_idx])

idx_loader_train = Data.DataLoader(train_idx, batch_size=batch_size, shuffle=True)
idx_loader_val = Data.DataLoader(val_idx, batch_size=batch_size)
idx_loader_test = Data.DataLoader(test_idx, batch_size=batch_size)
idx_loader = Data.DataLoader(doc_idx, batch_size=batch_size, shuffle=True)

(8654, 300) (8654, 2) (2404, 300) (2404, 2) (35868, 300) (35868, 2)
38272


/tmp/ipykernel_1355/3238164346.py:132: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(mask, dtype=np.bool)
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expe

graph information:
graph information:
graph information:
Graph(num_nodes=38272, num_edges=24734470,
      ndata_schemes={'input_ids': Scheme(shape=(64,), dtype=torch.int64), 'attention_mask': Scheme(shape=(64,), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64), 'train': Scheme(shape=(), dtype=torch.float32), 'val': Scheme(shape=(), dtype=torch.float32), 'test': Scheme(shape=(), dtype=torch.float32), 'label_train': Scheme(shape=(), dtype=torch.int64), 'cls_feats': Scheme(shape=(768,), dtype=torch.float32)}
      edata_schemes={'edge_weight': Scheme(shape=(), dtype=torch.float32)})
Graph(num_nodes=38272, num_edges=24734470,
      ndata_schemes={'input_ids': Scheme(shape=(64,), dtype=torch.int64), 'attention_mask': Scheme(shape=(64,), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64), 'train': Scheme(shape=(), dtype=torch.float32), 'val': Scheme(shape=(), dtype=torch.float32), 'test': Scheme(shape=(), dtype=torch.float32), 'label_train': Scheme(shape=(), dty

In [29]:
class F1Score(Metric):

    def __init__(self, *args, **kwargs):
        self.f1 = 0
        self.count = 0
        super().__init__(*args, **kwargs)

    def update(self, output):
        y_pred, y = output[0].detach(), output[1].detach()

        _, predicted = torch.max(y_pred, 1)
        f = f1_score(y.cpu(), predicted.cpu(), average='micro')
        self.f1 += f
        self.count += 1

    def reset(self):
        self.f1 = 0
        self.count = 0
        super(F1Score, self).reset()

    def compute(self):
        return self.f1 / self.count

In [30]:
import wandb 

def update_feature():
    global model, g, doc_mask
    dataloader = Data.DataLoader(
        Data.TensorDataset(g.ndata['input_ids'][doc_mask], g.ndata['attention_mask'][doc_mask]),
        batch_size=1024
    )
    with torch.no_grad():
        model = model.to(gpu)
        model.eval()
        cls_list = []
        for i, batch in enumerate(dataloader):
            input_ids, attention_mask = [x.to(gpu) for x in batch]
            output = model.bert_model(input_ids=input_ids, attention_mask=attention_mask)[0][:, 0]
            cls_list.append(output.cpu())
        cls_feat = torch.cat(cls_list, axis=0)
    g = g.to(cpu)
    g.ndata['cls_feats'][doc_mask] = cls_feat
    return g


optimizer = torch.optim.Adam([
        {'params': model.bert_model.parameters(), 'lr': bert_lr},
        {'params': model.classifier.parameters(), 'lr': bert_lr},
        {'params': model.gcn.parameters(), 'lr': gcn_lr},
    ], lr=gcn_lr
)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)


def train_step(engine, batch):
    global model, g, optimizer
    model.train()
    model = model.to(gpu)
    g = g.to(gpu)
    optimizer.zero_grad()
    # print("batch",batch)
    (idx, ) = [x.to(gpu) for x in batch]
    optimizer.zero_grad()
    train_mask = g.ndata['train'][idx].type(torch.BoolTensor)
    y_pred = model(g, idx)[train_mask]
    y_true = g.ndata['label_train'][idx][train_mask]
    loss = F.nll_loss(y_pred, y_true)
    loss.backward()
    optimizer.step()
    g.ndata['cls_feats'].detach_()
    train_loss = loss.item()
    with torch.no_grad():
        if train_mask.sum() > 0:
            y_true = y_true.detach().cpu()
            y_pred = y_pred.argmax(axis=1).detach().cpu()
            train_acc = accuracy_score(y_true, y_pred)
        else:
            train_acc = 1
    return train_loss, train_acc


trainer = Engine(train_step)
pbar = ProgressBar()
pbar.attach(trainer)

@trainer.on(Events.EPOCH_COMPLETED)
def reset_graph(trainer):
    scheduler.step()
    update_feature()
    torch.cuda.empty_cache()

count = 0 
def test_step(engine, batch):
    global model, g
    with torch.no_grad():
        model.eval()
        model = model.to(gpu)
        g = g.to(gpu)
        (idx, ) = [x.to(gpu) for x in batch]
        y_pred = model(g, idx)
        y_true = g.ndata['label'][idx]
        # wandb.init(project="embedding")
        wandb.init(settings=wandb.Settings(start_method="thread"))


        df_data = {
                    'target':y_true.tolist(),
                    'feature': y_pred.tolist()
                }
        df = pd.DataFrame(df_data)
        wandb.log({"Embedding": df})
        wandb.finish()
         
        return y_pred, y_true


evaluator = Engine(test_step)
eval_pbar = ProgressBar()
eval_pbar.attach(evaluator)
metrics={
    'acc': Accuracy(),
    'nll': Loss(torch.nn.NLLLoss()),
    'f1' : F1Score()
}
for name, function in metrics.items():
    function.attach(evaluator, name)

In [31]:
# @trainer.on(Events.EPOCH_COMPLETED)

g = update_feature()
trainer.run(idx_loader, max_epochs=nb_epochs)

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

...

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

[1/47]   2%|2          [00:00<?]

State:
	iteration: 470
	epoch: 10
	epoch_length: 47
	max_epochs: 10
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [ ]:
import pandas as pd 
evaluator.run(idx_loader_train)

[1/34]   3%|2          [00:00<?]

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


In [ ]:
# @trainer.on(Events.EPOCH_COMPLETED)
# def log_training_results(trainer):
#     evaluator.run(idx_loader_train)
#     metrics = evaluator.state.metrics
#     train_acc, train_nll = metrics["acc"], metrics["nll"]
#     evaluator.run(idx_loader_val)
#     metrics = evaluator.state.metrics
#     val_acc, val_nll, val_f1 = metrics["acc"], metrics["nll"], metrics["f1"]
#     evaluator.run(idx_loader_test)
#     metrics = evaluator.state.metrics
#     test_acc, test_nll, test_f1 = metrics["acc"], metrics["nll"], metrics["f1"]
#     logger.info(
#         "Epoch: {}  Train acc: {:.4f} loss: {:.4f}  Val acc: {:.4f} loss: {:.4f} f1: {:.4f}  Test acc: {:.4f} loss: {:.4f} f1: {:.4f}"
#         .format(trainer.state.epoch, train_acc, train_nll, 
#                 val_acc, val_nll, val_f1, 
#                 test_acc, test_nll, test_f1
#                )
#     )
#     if test_f1 > log_training_results.best_test_f1 and test_f1 > 0.8712:
#         logger.info("New checkpoint")
#         torch.save(
#             {
#                 'bert_model': model.bert_model.state_dict(),
#                 'classifier': model.classifier.state_dict(),
#                 'gcn': model.gcn.state_dict(),
#                 'optimizer': optimizer.state_dict(),
#                 'epoch': trainer.state.epoch,
#                 'seed':trainer.state.seed,
#             },
#             os.path.join(
#                 ckpt_dir, 'checkpoint.pth'
#             )
#         )
#         log_training_results.best_test_f1 = test_f1


# log_training_results.best_test_f1 = 0



In [16]:
# @trainer.on(Events.EPOCH_COMPLETED)
# def log_training_results(trainer):
#     evaluator.run(idx_loader_train)
#     metrics = evaluator.state.metrics
#     train_acc, train_nll = metrics["acc"], metrics["nll"]
#     evaluator.run(idx_loader_val)
#     metrics = evaluator.state.metrics
#     val_acc, val_nll, val_f1 = metrics["acc"], metrics["nll"], metrics["f1"]
#     evaluator.run(idx_loader_test)
#     metrics = evaluator.state.metrics
#     test_acc, test_nll, test_f1 = metrics["acc"], metrics["nll"], metrics["f1"]
#     logger.info(
#         "Epoch: {}  Train acc: {:.4f} loss: {:.4f}  Val acc: {:.4f} loss: {:.4f} f1: {:.4f}  Test acc: {:.4f} loss: {:.4f} f1: {:.4f}"
#         .format(trainer.state.epoch, train_acc, train_nll, 
#                 val_acc, val_nll, val_f1, 
#                 test_acc, test_nll, test_f1
#                )
#     )
#     if test_f1 > log_training_results.best_test_f1 and test_f1 > 0.8712:
#         logger.info("New checkpoint")
#         torch.save(
#             {
#                 'bert_model': model.bert_model.state_dict(),
#                 'classifier': model.classifier.state_dict(),
#                 'gcn': model.gcn.state_dict(),
#                 'optimizer': optimizer.state_dict(),
#                 'epoch': trainer.state.epoch,
#                 'seed':trainer.state.seed,
#             },
#             os.path.join(
#                 ckpt_dir, 'checkpoint.pth'
#             )
#         )
#         log_training_results.best_test_f1 = test_f1


# log_training_results.best_test_f1 = 0
# g = update_feature()
# trainer.run(idx_loader, max_epochs=nb_epochs)

In [17]:
dataloader = Data.DataLoader(
    Data.TensorDataset(g.ndata['input_ids'][doc_mask], g.ndata['attention_mask'][doc_mask]),
    batch_size=1024
)
for i, batch in enumerate(dataloader):
    input_ids, attention_mask = [x.to(gpu) for x in batch]
    print(input_ids)

tensor([[    0,  7761,  6821,  ...,  3714, 11540,     2],
        [    0,  7761,   385,  ...,  3137,    36,     2],
        [    0,  7761,    40,  ...,  8493,   155,     2],
        ...,
        [    0,  7761,   475,  ...,  1115,   338,     2],
        [    0,  7761,  1236,  ...,   257,    36,     2],
        [    0,  7761,  7321,  ...,  1153,   206,     2]], device='cuda:0')
tensor([[    0,  7761, 13145,  ...,  8060,   295,     2],
        [    0,  7761, 45584,  ...,   910, 28024,     2],
        [    0,  7761,  3778,  ...,   257,   875,     2],
        ...,
        [    0,  7761,  6821,  ...,  2527,  1717,     2],
        [    0,  7761,  3553,  ...,   260,  1891,     2],
        [    0,  7761,  2242,  ...,   475, 10570,     2]], device='cuda:0')


In [18]:
doc_mask

array([ True,  True,  True, ...,  True,  True,  True])

In [19]:
global model, g , doc_mask
# dataloader = Data.DataLoader(
#     Data.TensorDataset(g.ndata['input_ids'][doc_mask], g.ndata['attention_mask'][doc_mask]),
    
#     batch_size=1024
# )

idx_loader_train = Data.DataLoader(train_idx, batch_size=batch_size, shuffle=True)

with torch.no_grad():
    global model, g 
    model.eval()
    model = model.to(gpu)
    
    g = g.to(gpu)
    cls_list = []
    for i, batch in enumerate(idx_loader_train):
        idx, attention_mask = [x.to(gpu) for x in batch]
        model = model.to(gpu)
        g = g.to(gpu)
        # (idx, ) = [x.to(gpu) for x in batch]
        print(g,np.shape(idx))
        y_pred = model(g, idx)
        y_true = g.ndata['label'][idx]
        
        df_data = {
                    'target':y_true.to_lost(),
                    'feature': y_pred.to_lost()
                }
        df = pd.DataFrame(df_data)
        wandb.log({"Embedding": df})
        wandb.finish()
        
        
        # y_pred = model(g, idx)
        # y_true = g.ndata['label'][idx]
        # y = y_true.tolist()
        # # output = model(features, adj)
        # wandb.init(project="embedding_3")
        # # out = output[idx_train].tolist()
        # out = y_pred.tolist()
        # df_data = {
        #             'target':y,
        #             'feature': out
        #         }
        # df = pd.DataFrame(df_data)
        # wandb.log({"Embedding": df})
        # wandb.finish()


ValueError: not enough values to unpack (expected 2, got 1)

# BERTGAT